## 文档的创建和实例

In [3]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD

In [4]:
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [5]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 引用字段保存的特殊情况

注意下述引用字段，保存时的特性：即引用字段修改值后，是否能否同步更新原集合。关注 2.3.1-2文档中的`save()`函数的 `cascade` 参数。

### 引用字段 Reference fields

In [6]:
class BookAuthor(Document):
    name = StringField()
    age = StringField()
    meta = {"db_alias": TEST_DB1}

In [17]:
class BookReferenceAuthor(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.3-5")
    author = ReferenceField(BookAuthor)  # 其他文档作为参数
    meta = {"db_alias": TEST_DB1}

In [18]:
author = BookAuthor(name="司马迁", age="西汉").save()
book_a = BookReferenceAuthor(bookid="wx93ll7ppa", name="史记", author=author)
book_a.save()

<BookReferenceAuthor: BookReferenceAuthor object>

mongodb储存结果:
```json
{
    "_id" : ObjectId("63fe11b26afa912dc75ba521"),
    "bookid" : "wx93ll7ppa",
    "name" : "史记",
    "chapter" : "2.3-2.4",
    "author" : ObjectId("63fe11b26afa912dc75ba520")
}
```
即引用的文档是以 ObjectId 形式储存的。

In [23]:
# 被引用文档已经保存至数据库时的引用操作
author_b = BookAuthor.objects(name="蒲松龄").first()
book_b = BookReferenceAuthor(bookid="wx932wepa", name="聊斋志异", author=author_b)
book_b.save()

<BookReferenceAuthor: BookReferenceAuthor object>

### ListFields中的多对多引用(Many to Many with ListFields)

In [31]:
class BookReferenceMany(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.3-5")
    authors = ListField(ReferenceField(BookAuthor))  # 其他文档作为参数
    meta = {"db_alias": TEST_DB1}

In [33]:
author_c = BookAuthor.objects(name="司马迁").first()
author_d = BookAuthor.objects(name="蒲松龄").first()
book_b = BookReferenceMany(bookid="wx9322pmcpa", name="聊斋志异史记", authors=[author_c, author_d])
book_b.save()

<BookReferenceMany: BookReferenceMany object>

### 其他应用模式?只引用文档的iD?如何关联查询?

### 引用文档删除 

By default, MongoDB doesn’t check the integrity of your data, so deleting documents that other documents still hold references to will lead to consistency issues. Mongoengine’s ReferenceField adds some functionality to safeguard against these kinds of database integrity problems, providing each reference with a delete rule specification. A delete rule is specified by supplying the reverse_delete_rule attributes on the ReferenceField definition, like this:


In [9]:
from mongoengine import CASCADE
class BookReferenceDelete(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.3-5")
    author = ReferenceField(BookAuthor, reverse_delete_rule=CASCADE)  # 其他文档作为参数
    meta = {"db_alias": TEST_DB1}

The declaration in this example means that when an `BookAuthor` object is removed, the `BookReferenceDelete` that references that `author` is removed as well. If a whole batch of `authors` is removed, all BookReferenceDelete pages that are linked are removed as well.

In [11]:
author_del = BookAuthor(name="吴敬梓").save()
book_del = BookReferenceDelete(bookid="wx93abpmcpa", name="儒林外史", author=author_del)
book_del.save()

<BookReferenceDelete: BookReferenceDelete object>

BookReferenceDelete集合的保存结果:
```json
{
    "_id" : ObjectId("63fea50affbc7d464ef9a543"),
    "bookid" : "wx93abpmcpa",
    "name" : "儒林外史",
    "chapter" : "2.3.3.3-5",
    "author" : ObjectId("63fea50affbc7d464ef9a542")
}
```
BookAuthor集合:
```json
{
    "_id" : ObjectId("63fea50affbc7d464ef9a542"),
    "name" : "吴敬梓"
}
```

In [14]:
# 删除 被引用的文档  ObjectId("63fea50affbc7d464ef9a542")
BookAuthor.objects(name="吴敬梓").delete()
# 1
# 删除成功

0

In [15]:
# 查询 儒林外史 文档
BookReferenceDelete.objects(bookid="wx93abpmcpa", name="儒林外史").first()
# 由于设置  reverse_delete_rule=CASCADE， 删除作者后，引用的文档 儒林外史  文档也被删除。
# 此查询不返回值

#### 关于 引用字段删除的中文翻译和主要参数

默认情况下，MongoDB不检查数据的完整性，因此删除其他文档仍然存在引用的文档将导致一致性问题。Mongoengine 的ReferenceField增加了一些功能来防范这些类型的数据库完整性问题，为每个引用提供删除规则规范。通过reverse_delete_rule在ReferenceField定义上提供属性 来指定删除规则，如下所示

它的值可以采用以下任何常量：

mongoengine.DO_NOTHING
这是默认设置，不会执行任何操作。删除速度很快，但可能会导致数据库不一致或悬空引用。

mongoengine.DENY
如果仍存在对被删除对象的引用，则拒绝删除。

mongoengine.NULLIFY
删除任何仍然指向被删除对象的对象字段（使用MongoDB的“未设置”操作），从而有效地消除关系。
即删除了被引用的字段，对引用它的字段无影响，举个例子，假如，文章的作者字段采用的是引用字段，那么作者一旦被删除，那么，由他写的文章仅仅是没有了作者，他的文章都还在。

mongoengine.CASCADE
引用字段被删除，则引用此字段的文档也会被删除，
举个例子，假如，文章的作者字段采用的是引用字段，那么作者一旦被删除，那么，由他写的文章也都被删除。

mongoengine.PULL
从ListField（ReferenceField）的任何对象的字段中删除对该对象的引用（使用MongoDB的“拉”操作 ）。

### 其他引用字段

```python
class mongoengine.fields.LazyReferenceField( document_type , passthrough = False , dbref = False , reverse_delete_rule = 0 , ** kwargs )
```
对文档的真正懒惰引用。与它不同的是，`ReferenceField`它不会在访问时自动（懒惰地）取消引用。相反，access `LazyReferenceField` 将返回一个类实例，允许访问pk或通过使用方法`fecth()`手动取消引用。

初始化参考字段。

参数:   
    `dbref` – 将引用存储为`DBRef` 或作为 `ObjectId.id` 。  
    `reverse_delete_rule` – 确定删除引用对象时要执行的操作  
    `passthrough`——尝试访问未知字段时，`LazyReference`实例将自动调用`fetch()`并尝试检索已获取文档中的字段。请注意，这仅适用于获取字段（而不是设置或删除）。  

url: https://mongoengine-odm.readthedocs.io/apireference.html#mongoengine.fields.GenericReferenceField
`classmongoengine.fields.GenericReferenceField(*args, **kwargs)`
A reference to any Document subclass that will be automatically dereferenced on access (lazily).

Note this field works the same way as ReferenceField, doing database I/O access the first time it is accessed (even if it’s to access it pk or id field). To solve this you should consider using the GenericLazyReferenceField.

Note

Any documents used as a generic reference must be registered in the document registry. Importing the model will automatically register it.

You can use the choices param to limit the acceptable Document types

Parameters  

    db_field – The database field to store this field in (defaults to the name of the field)

    required – If the field is required. Whether it has to have a value or not. Defaults to False.

    default – (optional) The default value for this field if no value has been set, if the value is set to None or has been unset. It can be a callable.

    unique – Is the field value unique or not (Creates an index). Defaults to False.

    unique_with – (optional) The other field this field should be unique with (Creates an index).

    primary_key – Mark this field as the primary key ((Creates an index)). Defaults to False.

    validation – (optional) A callable to validate the value of the field. The callable takes the value as parameter and should raise a ValidationError if validation fails

    choices – (optional) The valid choices

    null – (optional) If the field value can be null when a default exist. If not set, the default value

will be used in case a field with a default value is set to None. Defaults to False. :param sparse: (optional) sparse=True combined with unique=True and required=False

means that uniqueness won’t be enforced for None values (Creates an index). Defaults to False.

Parameters
**kwargs –

(optional) Arbitrary indirection-free metadata for this field can be supplied as additional keyword arguments and accessed as attributes of the field. Must not conflict with any existing attributes. Common metadata includes verbose_name and help_text.


url:https://mongoengine-odm.readthedocs.io/apireference.html#mongoengine.fields.GenericLazyReferenceField

`classmongoengine.fields.GenericLazyReferenceField(*args, **kwargs)`  
A reference to any Document subclass. Unlike the GenericReferenceField it will not be automatically (lazily) dereferenced on access. Instead, access will return a LazyReference class instance, allowing access to pk or manual dereference by using fetch() method.

Note

Any documents used as a generic reference must be registered in the document registry. Importing the model will automatically register it.

You can use the choices param to limit the acceptable Document types

Parameters  
    
    db_field – The database field to store this field in (defaults to the name of the field)

    required – If the field is required. Whether it has to have a value or not. Defaults to False.

    default – (optional) The default value for this field if no value has been set, if the value is set to None or has been unset. It can be a callable.

    unique – Is the field value unique or not (Creates an index). Defaults to False.

    unique_with – (optional) The other field this field should be unique with (Creates an index).

    primary_key – Mark this field as the primary key ((Creates an index)). Defaults to False.

    validation – (optional) A callable to validate the value of the field. The callable takes the value as parameter and should raise a ValidationError if validation fails

    choices – (optional) The valid choices

    null – (optional) If the field value can be null when a default exist. If not set, the default value

will be used in case a field with a default value is set to None. Defaults to False. :param sparse: (optional) sparse=True combined with unique=True and required=False

means that uniqueness won’t be enforced for None values (Creates an index). Defaults to False.

Parameters
**kwargs –

(optional) Arbitrary indirection-free metadata for this field can be supplied as additional keyword arguments and accessed as attributes of the field. Must not conflict with any existing attributes. Common metadata includes verbose_name and help_text.


url:
https://mongoengine-odm.readthedocs.io/apireference.html#mongoengine.fields.CachedReferenceField

`classmongoengine.fields.CachedReferenceField(document_type, fields=None, auto_sync=True, **kwargs)`  
A referencefield with cache fields to purpose pseudo-joins

Initialises the Cached Reference Field.

Parameters  
    
    document_type – The type of Document that will be referenced

    fields – A list of fields to be cached in document

    auto_sync – if True documents are auto updated

    kwargs – Keyword arguments passed into the parent BaseField